In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
train_clientes = pd.read_csv('_data/train_clientes.csv')
train_requerimientos = pd.read_csv('_data/train_requerimientos.csv')
test_clientes = pd.read_csv('_data/test_clientes.csv')
test_requerimientos = pd.read_csv('_data/test_requerimientos.csv')

In [3]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import time

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold

/home/robert/anaconda3/envs/kaggle/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/robert/anaconda3/envs/kaggle/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.4 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [391]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [5]:
results = pd.read_csv('_data/results.csv')

In [15]:
results2 = results[results['time'].apply(lambda x: False)].copy()

In [333]:
def run_model(model, X, y, dataset_desc='', params_desc=''):
    model_name = str(model.__class__).split('.')[-1].replace('>','').replace("'",'')
    print(model_name, end=' ')
    auc = []
    logloss = []
    start = int(time.time() * 1000)
    for train_idx, test_idx in kfolds.split(X, y):
        model.fit(X.iloc[train_idx,:], y[train_idx])
        y_ = model.predict_proba(X.iloc[test_idx,:])[:,1]
        auc += [roc_auc_score(y.iloc[test_idx], y_)]
        logloss += [log_loss(y.iloc[test_idx], y_)]
    end = int(time.time() * 1000)
    results2.loc[len(results2)] = [dataset_desc, model_name, params_desc,
                                 np.mean(auc), np.std(auc), 
                                 np.mean(logloss), np.std(logloss), str(end - start)]
    print(np.mean(auc), np.mean(logloss))
#     winsound.Beep(2500, 1000)

In [17]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',150)

In [18]:
train = pd.read_csv('_data/train_independent.csv')
test = pd.read_csv('_data/test_independent.csv')

In [19]:
train.head()

,ID_CORRELATIVO,FLG_BANCARIZADO,RANG_INGRESO,FLAG_LIMA_PROVINCIA,EDAD,ANTIGUEDAD,ATTRITION,RANG_SDO_PASIVO_MENOS0,SDO_ACTIVO_MENOS0,SDO_ACTIVO_MENOS1,SDO_ACTIVO_MENOS2,SDO_ACTIVO_MENOS3,SDO_ACTIVO_MENOS4,SDO_ACTIVO_MENOS5,FLG_SEGURO_MENOS0,FLG_SEGURO_MENOS1,FLG_SEGURO_MENOS2,FLG_SEGURO_MENOS3,FLG_SEGURO_MENOS4,FLG_SEGURO_MENOS5,RANG_NRO_PRODUCTOS_MENOS0,FLG_NOMINA,NRO_ACCES_CANAL1_MENOS0,NRO_ACCES_CANAL1_MENOS1,NRO_ACCES_CANAL1_MENOS2,NRO_ACCES_CANAL1_MENOS3,NRO_ACCES_CANAL1_MENOS4,NRO_ACCES_CANAL1_MENOS5,NRO_ACCES_CANAL2_MENOS0,NRO_ACCES_CANAL2_MENOS1,NRO_ACCES_CANAL2_MENOS2,NRO_ACCES_CANAL2_MENOS3,NRO_ACCES_CANAL2_MENOS4,NRO_ACCES_CANAL2_MENOS5,NRO_ACCES_CANAL3_MENOS0,NRO_ACCES_CANAL3_MENOS1,NRO_ACCES_CANAL3_MENOS2,NRO_ACCES_CANAL3_MENOS3,NRO_ACCES_CANAL3_MENOS4,NRO_ACCES_CANAL3_MENOS5,NRO_ENTID_SSFF_MENOS0,NRO_ENTID_SSFF_MENOS1,NRO_ENTID_SSFF_MENOS2,NRO_ENTID_SSFF_MENOS3,NRO_ENTID_SSFF_MENOS4,NRO_ENTID_SSFF_MENOS5,FLG_SDO_OTSSFF_MENOS0,FLG_SDO_OTSSFF_MENOS1,FLG_SDO_OTSSFF_MENOS2,FLG_SDO_OTSSFF_MENOS3,FLG_SDO_OTSSFF_MENOS4,FLG_SDO_OTSSFF_MENOS5,TIPO_REQUERIMIENTO2-Reclamo,TIPO_REQUERIMIENTO2-Solicitud,DICTAMEN-NO PROCEDE,DICTAMEN-PROCEDE TOTAL,DICTAMEN-PROCEDE PARCIAL,PRODUCTO_SERVICIO_2-Producto 20,PRODUCTO_SERVICIO_2-Producto 17,PRODUCTO_SERVICIO_2-Producto 07,PRODUCTO_SERVICIO_2-Producto 18,PRODUCTO_SERVICIO_2-Producto 21,PRODUCTO_SERVICIO_2-Producto 01,PRODUCTO_SERVICIO_2-Producto 05,CODMES-201206,CODMES-201205,CODMES-201208,CODMES-201204,CODMES-201203,CODMES-201207,EDAD-isnull,ANTIGUEDAD-isnull,REQUERIMIENTO-isnull
0,35653,1.0,0.666667,0.5,0.081633,0.6,0,0.642857,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.75,0.0,0.142857,0.071429,0.0,0.142857,0.307692,0.076923,0.380952,0.5,0.7,0.6,0.4,0.6,0.090909,0.000000,0.032258,0.1875,0.064516,0.15625,0.285714,0.142857,0.142857,0.142857,0.142857,0.142857,1.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,66575,1.0,0.333333,1.0,0.122449,0.0,0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.727273,0.000000,0.000000,0.0000,0.000000,0.00000,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,56800,1.0,0.111111,1.0,0.265306,0.4,0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.2500,0.000000,0.06250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,1.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8410,1.0,0.444444,1.0,0.857143,0.5,0,0.214286,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.50,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.028571,0.032258,0.0000,0.064516,0.00000,0.428571,0.428571,0.428571,0.428571,0.428571,0.428571,1.0,1.0,1.0,1.0,1.0,1.0,0.00,0.5,0.333333,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
4,6853,1.0,0.000000,0.5,0.081633,0.0,0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
run_model(LGBMClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'], 
                  'independent', 'default')

LGBMClassifier 0.85678083986 0.301999011848


In [25]:
results.sort_values('logloss').head()

,dataset,model,params,aucroc,aucroc-std,logloss,logloss-std,time
16,clipped,CatBoostClassifier,default,0.856796,0.001835,0.302478,0.001686,558997
24,norm,CatBoostClassifier,default,0.856575,0.001992,0.302597,0.001660,571138
2,preprocessed,CatBoostClassifier,default,0.856396,0.001619,0.303038,0.001421,393646
5,merge,CatBoostClassifier,default,0.855920,0.001691,0.303143,0.001529,607605
8,full,CatBoostClassifier,default,0.855452,0.001837,0.303650,0.001699,756572


In [24]:
results2

,dataset,model,params,aucroc,aucroc-std,logloss,logloss-std,time
0,independent,LGBMClassifier,default,0.856781,0.001575,0.301999,0.001463,7477


In [26]:
train.shape

(70000, 73)

red: constraints the model
     increase it to impede fitting
     increase it to reduce overfitting
     decrease it to allow model fit easier
green: increase it to better fit
       increase it to reduce underfitting
       decrease it if overfits

In [392]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 100,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'colsample_bytree': 0.8,
    
#     'min_child_weight': 0.6,    
#     'learning_rate': 0.1,    
#     'max_depth': 70,    
#     'class_weight': 'balanced',
#     'n_estimators': 120
}

In [393]:
LGBMClassifier(**params)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.1, max_depth=-1, metric='binary_logloss',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=100, objective='binary',
        random_state=None, reg_alpha=1, reg_lambda=1, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [394]:
run_model(LGBMClassifier(**params), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'], 
                  'independent', 'custom')

LGBMClassifier 0.85994711954 0.298089376992


In [397]:
results2.tail(10)

,dataset,model,params,aucroc,aucroc-std,logloss,logloss-std,time
80,independent,LGBMClassifier,custom,0.859000,0.005580,0.299001,0.004677,10930
81,independent,LGBMClassifier,custom,0.858036,0.005330,0.299707,0.004506,11532
82,independent,LGBMClassifier,custom,0.858847,0.005351,0.299230,0.004565,10730
83,independent,LGBMClassifier,custom,0.858740,0.005552,0.299179,0.004665,10642
84,independent,LGBMClassifier,custom,0.859000,0.005580,0.299001,0.004677,10609
85,independent,LGBMClassifier,custom,0.857670,0.005487,0.300748,0.004665,10352
86,independent,LGBMClassifier,custom,0.859000,0.005580,0.299001,0.004677,10647
87,independent,LGBMClassifier,custom,0.859518,0.005983,0.298694,0.004962,10887
88,independent,LGBMClassifier,custom,0.859247,0.002646,0.298785,0.002695,10523
89,independent,LGBMClassifier,custom,0.859947,0.001694,0.298089,0.001655,9301


## Submission

In [ ]:
run_model(LGBMClassifier(), train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'], 
                  'independent', 'default')

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 100,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'colsample_bytree': 0.8,
}

In [398]:
model = LGBMClassifier(**params)

In [399]:
model.fit(train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), train['ATTRITION'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.1, max_depth=-1, metric='binary_logloss',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=100, objective='binary',
        random_state=None, reg_alpha=1, reg_lambda=1, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [400]:
y_test_pred = model.predict_proba(test.drop('ID_CORRELATIVO', axis=1))[:,1]
submission = pd.DataFrame()
submission['ID_CORRELATIVO'] = test['ID_CORRELATIVO']
submission['ATTRITION'] = y_test_pred

In [401]:
submission.head()

,ID_CORRELATIVO,ATTRITION
0,47411,0.276445
1,39861,0.216979
2,38898,0.021382
3,50927,0.019478
4,32969,0.349304


In [402]:
submission.to_csv('_data/submission2.csv', index=False)